In [1]:
import pandas as pd

In [2]:
pts = pd.read_csv("predictions/ppr_estimate.csv")

In [3]:
pts.head()

,player,pid,Season,misc_fpts,lasso,ols,rf,ridge,median_est,delta,pos
0,A.J. Brown (PHI),18218,2024,289.6,289.526548,289.063789,288.354296,289.064135,289.063962,-0.536038,wr
1,A.T. Perry (NO),24353,2024,60.6,60.614354,60.455730,60.354214,60.459152,60.457441,-0.142559,wr
2,AJ Barner (SEA),26335,2024,0.0,-0.004305,-0.009750,-0.000034,-0.009788,-0.007028,-0.007028,te
3,AJ Dillon (GB),19358,2024,117.6,117.657398,117.000441,117.747178,117.004988,117.331193,-0.268807,rb
4,AJ McCarron (FA),12091,2024,0.8,0.805871,0.656989,0.912772,0.659277,0.732574,-0.067426,qb


In [8]:
pts = pts.sort_values("median_est", ascending=False).reset_index(drop=True)

In [17]:
dst_projections = pd.read_csv("data/dst_projections.csv")
k_projections = pd.read_csv("data/k_projections.csv")

In [15]:
def clean_dstk(df: pd.DataFrame):
    import numpy as np

    df = df[["player", "pid", "Season", "fpts", "pos"]]
    df["misc_fpts"] = df["fpts"]
    df["lasso"] = df["fpts"]
    df["ols"] = df["fpts"]
    df["rf"] = df["fpts"]
    df["ridge"] = df["fpts"]
    df["median_est"] = df["fpts"]
    df["delta"] = 0

    return df[[
        "player",
        "pid",
        "Season",
        "misc_fpts",
        "lasso",
        "ols",
        "rf",
        "ridge",
        "median_est",
        "delta",
        "pos",
    ]]


In [19]:
pts = pd.concat([pts, clean_dstk(dst_projections), clean_dstk(k_projections)])

/var/folders/6h/4bw0dlhd627__5j16_g7ndwh0000gn/T/ipykernel_50156/2783814974.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["misc_fpts"] = df["fpts"]
/var/folders/6h/4bw0dlhd627__5j16_g7ndwh0000gn/T/ipykernel_50156/2783814974.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lasso"] = df["fpts"]
/var/folders/6h/4bw0dlhd627__5j16_g7ndwh0000gn/T/ipykernel_50156/2783814974.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [20]:
pts = pts.sort_values("median_est", ascending=False).reset_index(drop=True)

In [21]:
pts.head()

,player,pid,Season,misc_fpts,lasso,ols,rf,ridge,median_est,delta,pos
0,Josh Allen (BUF),17298,2024,410.9,411.099312,412.190850,396.613602,412.211819,411.645081,0.745081,qb
1,CeeDee Lamb (DAL),19202,2024,403.2,403.229977,403.562106,391.941219,403.566691,403.396041,0.196041,wr
2,Christian McCaffrey (SF),16393,2024,391.3,391.703509,390.235317,319.027046,390.275056,390.255187,-1.044813,rb
3,Tyreek Hill (MIA),15802,2024,376.4,376.355253,375.772263,377.116938,375.782719,376.068986,-0.331014,wr
4,Jalen Hurts (PHI),19275,2024,371.9,372.060432,369.021016,368.298160,369.082277,369.051646,-2.848354,qb


In [22]:
pts.to_csv('predictions/draft_2024_order.csv', index=False)

#### calc VORP

In [25]:
qbs = pts.query("pos == 'qb'")
qbs = qbs.assign(rank=range(len(qbs)))
qbst = qbs[qbs['rank']==9]
qbs['vorp'] = qbs['median_est'] - qbst['median_est'].values

In [37]:
flex = pts.query("pos in ['rb', 'wr', 'te']")
flex = flex.assign(rank=range(len(flex)))
flex = flex[flex['rank']==(5*10-1)]
flex['flex_vorp'] = flex['median_est'] - flex['median_est'].values

In [44]:
def calc_vorp(pts_df: pd.DataFrame, pos: str, num: int):
    vorpdf = pts.query(f"pos == '{pos}'")
    vorpdf = vorpdf.assign(rank=range(len(vorpdf)))
    vropmax = vorpdf[vorpdf['rank']==(num*9-1)]
    vorpdf['vorp'] = vorpdf['median_est'] - vropmax['median_est'].values

    return vorpdf
    

In [45]:
vorps = pd.DataFrame()
for p, n in [('qb', 1), ('rb', 2), ('wr', 2), ('te', 1), ('dst', 1), ('k', 1)]:
    vorpdf = calc_vorp(pts, p, n)

    vorps = pd.concat([vorps, vorpdf], axis=0)

In [48]:
ests_vorps = vorps.merge(flex[['pid', 'flex_vorp']], how='left', on='pid')


In [ ]:
ests_vorps.to_csv('predictions/pts_vorps.csv', index=False)

In [ ]:
vorps